In [1]:
import numpy as np
import matplotlib.pyplot as plt
import autokeras as ak
import tensorflow as tf
import os

In [2]:
import logging

logging.basicConfig(level = logging.INFO)

gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     logging.info(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
except RuntimeError as e:
# Visible devices must be set before GPUs have been initialized
    logging.info(e)

In [3]:
exposure = ''
data = np.load('/home/ML4NO/Data/n1000000{}_0910_classification.npz'.format(exposure))

In [4]:
data_selection = 1 # 0 for all, 1 for lowE(<5GeV), 2 for high(>5GeV)

if data_selection == 0:
    data_all = np.column_stack([data['ve_dune'], data['vu_dune'], data['vebar_dune'], data['vubar_dune']])
    directory = './models_all'
elif data_selection == 1:
    data_all = np.column_stack([data['ve_dune'][:,:36], data['vu_dune'][:,:36], data['vebar_dune'][:,:36], data['vubar_dune'][:,:36]])
    directory = './models_lowE'
elif data_selection == 2:
    data_all = np.column_stack([data['ve_dune'][:,36:], data['vu_dune'][:,36:], data['vebar_dune'][:,36:], data['vubar_dune'][:,36:]])
    directory = './models_highE'
target = data['cpv']

x_train = data_all[:10000]
y_train = target[:10000]
x_train2 = data_all[10000:900000]
y_train2 = target[10000:900000]
x_test = data_all[900000:]
y_test = target[900000:]

In [5]:
clf = ak.StructuredDataClassifier(overwrite=True, max_trials=10)
clf.fit(x_train, y_train,
           validation_split = 0.1,
           batch_size=64,
           epochs=20,
           verbose=1,
           shuffle = True
        )

Trial 10 Complete [00h 00m 57s]
val_accuracy: 0.7697368264198303

Best val_accuracy So Far: 0.9901315569877625
Total elapsed time: 00h 09m 50s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/20
157/157 [==============================] - 4s 15ms/step - loss: 0.5918 - accuracy: 0.6725
Epoch 2/20
157/157 [==============================] - 2s 16ms/step - loss: 0.2267 - accuracy: 0.9253
Epoch 3/20
157/157 [==============================] - 2s 16ms/step - loss: 0.1574 - accuracy: 0.9482
Epoch 4/20
157/157 [==============================] - 2s 15ms/step - loss: 0.1328 - accuracy: 0.9593
Epoch 5/20
157/157 [==============================] - 2s 15ms/step - loss: 0.1244 - accuracy: 0.9630
Epoch 6/20
157/157 [==============================] - 2s 16ms/step - loss: 0.1172 - accuracy: 0.9653
Epoch 7/20
157/157 [==============================] - 3s 16ms/step - loss: 0.1096 - accuracy: 0.9688
Epoch 8/20
157/157 [==============================] - 3s 17ms/step - loss: 0.1062 - accuracy: 0.9705
Epoch 9/20
157/157 [==============================] - 3s 17ms/step - loss: 0.0994 - accuracy: 0.9729
Epoch 10/20
157/157 [==============================] - 3s 17ms/step - loss: 0.0944 - accura


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


In [7]:
model = clf.export_model()
model.summary()
model.evaluate(x_test, y_test)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 264)]             0         
_________________________________________________________________
multi_category_encoding (Mul (None, 264)               0         
_________________________________________________________________
normalization (Normalization (None, 264)               529       
_________________________________________________________________
dense (Dense)                (None, 128)               33920     
_________________________________________________________________
re_lu (ReLU)                 (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                4128      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32)                0     

[0.09265825152397156, 0.9766600131988525]

In [10]:
model.fit(x_train2, y_train2,
           validation_split = 0.1,
           batch_size=64,
           epochs=5,
           verbose=1,
           shuffle = True
         )

Epoch 1/5
12516/12516 [==============================] - 209s 17ms/step - loss: 0.0601 - accuracy: 0.9854 - val_loss: 0.0480 - val_accuracy: 0.9883
Epoch 2/5
12516/12516 [==============================] - 206s 16ms/step - loss: 0.0480 - accuracy: 0.9887 - val_loss: 0.0786 - val_accuracy: 0.9775
Epoch 3/5
12516/12516 [==============================] - 209s 17ms/step - loss: 0.0432 - accuracy: 0.9900 - val_loss: 0.0344 - val_accuracy: 0.9915
Epoch 4/5
12516/12516 [==============================] - 213s 17ms/step - loss: 0.0399 - accuracy: 0.9909 - val_loss: 0.0337 - val_accuracy: 0.9934
Epoch 5/5
   60/12516 [..............................] - ETA: 3:12 - loss: 0.0194 - accuracy: 0.9961

KeyboardInterrupt: 

In [ ]:
index = 1
while os.path.isfile(directory + 'models/0910{}_{}.h5'.format(exposure,index)):
    index += 1
model.save(directory + 'models/0910{}_{}.h5'.format(exposure,index), overwrite=False)

In [ ]:
furthur_index = 1
path = directory + 'models_furthurTrain/0910_{}_{}'
while os.path.isdir(path.format(model_index, furthur_index)):
    furthur_index += 1

In [ ]:
scale_steps = np.logspace(-3, 0, 30)
before_train_loss = []
after_train_loss = []

for scale in scale_steps:
    x_train2_gen = np.random.normal(x_train2, np.sqrt(x_train2)*scale)
    x_test_gen = np.random.normal(x_test, np.sqrt(x_test)*scale)

    before_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

    model.fit(x_train2_gen, y_train2,
               validation_split = 0.1,
               callbacks=[tensorboard_callback],
               batch_size=64,
               epochs=5,
               verbose=1,
               shuffle = True
             )

    after_train_loss.append(model.evaluate(x_test_gen, y_test)[0])
    model.save(path.format(model_index, furthur_index) + '/std={}'.format(scale))

In [ ]:
scale_steps = np.logspace(-3, 0, 10)
before_train_loss = []
after_train_loss = []

for scale in scale_steps:
    x_train2_gen = np.random.normal(x_train2, np.sqrt(x_train2)*scale)
    x_test_gen = np.random.normal(x_test, np.sqrt(x_test)*scale)

    before_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

    model.fit(x_train2_gen, y_train2,
               validation_split = 0.1,
               batch_size=64,
               epochs=5,
               verbose=1,
               shuffle = True
             )
    after_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

In [ ]:
model_index = index
index = 1
path = directory + 'models_furthurTrain/0910{}_{}_{}.h5'
while os.path.isfile(path.format(exposure,model_index, index)):
    index += 1
model.save(path.format(exposure,model_index, index))
outfile = {'scale_steps': scale_steps,
           'before_train_loss': before_train_loss,
           'after_train_loss' :after_train_loss}
np.save(file = directory + 'models_furthurTrain/0910{}_{}_{}_result.npy'.format(exposure,model_index, index),
        arr = outfile)

In [ ]:
x_test_gen = np.random.poisson(x_test)

for i in range(5):
    x_train2_gen = np.random.poisson(x_train2)
    
    model.fit(x_train2_gen, y_train2,
              validation_split=0.1,
               batch_size=64,
               epochs=1,
               verbose=1,
               shuffle = True
             )
model.evaluate(x_test_gen, y_test)

In [ ]:
pred = model.predict(x_test_gen)

In [ ]:
cpc = np.where(y_test == 0)
cpv = np.where(y_test == 1)
plt.hist(pred[cpc], bins =100)
plt.hist(pred[cpv], bins = 100)
plt.show()

In [ ]:
furthur_index = index
index = 1
path = '/home/ML4NO/ML_DUNE/Classification/models_PoissonTrain/0910{}_{}_{}_{}.h5'
while os.path.isfile(path.format(exposure, model_index, furthur_index, index)):
    index += 1
model.save(path.format(exposure, model_index, furthur_index, index))